# 4. ResNet50

#### Garrett McCue

The next model architecture we will explore for classifying the images is the [ResNet50](https://arxiv.org/abs/1512.03385) architecture. The model architecture and pretrained weights from the ImageNet dataset can easily be downloaded and compiled from [Tensorflow and Keras](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19) for transfer learning applications.

The process of transfer learning for ConvNets includes:

1. Getting only the convolutional layers and associated pretrained weights of the model.
2. Freeze the weights of the pretrained convolutional layers
3. Attach Dense connected layers to the top
4. Train Dense top layers on the desired task. 

In [1]:
import os
# set correct local working directory and useful paths
ROOT = '/Users/garrettmccue/projects/cnn-alzheimers'
data_path = f'{ROOT}/data'
weights_path = f'{ROOT}/models/raw'

os.chdir(ROOT)
os.getcwd()

'/Users/garrettmccue/projects/cnn-alzheimers'

Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O  /Users/garrettmccue/projects/cnn-alzheimers/models/raw/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-07-18 20:02:41--  https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.190.144, 172.217.1.112, 142.250.191.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.190.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94765736 (90M) [application/octet-stream]
Saving to: ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5’

/Users/garrettmccue 100%[===================>]  90.38M  45.4MB/s    in 2.0s    

2022-07-18 20:02:43 (45.4 MB/s) - ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [94765736/94765736]



In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50

# set the downloaded weights file to a variable
local_weights_path = f'{weights_path}/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize base model from tensorflow.keras
# Set the input shape and remove dense layers (top), to match our weights file
# our image shapes are (256x256)
pre_trained_model = ResNet50(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)

# load pretrained weights
pre_trained_model.load_weights(local_weights_path)

# freeze weights of each layer
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

2022-07-18 20:02:53.284560: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

Using the all the pretrained layers we set the output from layer `conv5_block3_out` as the last_layer when passing it to the the build_resnet50() method. This method follows the same steps as the previous two models. 
1. loading and freezing the pretrained weights 
2. format the model ouput as the desired last layer
3. attach a dense network to the top for classification
4. compile the model w/ the Adam optimizer.

For brevity the steps described have been formatted and compiled into the resnet50.py models module. Below we can import the module and construct the full model model to be used. 

In [4]:
os.chdir(ROOT)

In [6]:
from src.models.resnet50 import build_resnet50

INPUT_SHAPE = (256,256,3)
CLASS_NUM = 5
LAST_LAYER = 'conv5_block3_out'

resnet50 = build_resnet50(input_shape=INPUT_SHAPE,
                            class_num=CLASS_NUM,
                            last_layer=LAST_LAYER,
                            dense_nodes=1024,
                            lr=0.001,
                            dropout=0.2,
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy', 'loss'],
                            print_summary = True)


Loading resnet50 weights from:
	 PATH = models/raw/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5...
Setting last layer as: conv5_block3_out
Last layer output shape: (None, 8, 8, 2048)
Building Dense layers...
Building Dense layers...
	Adding Dense layer with 1024 nodes
	Adding classification layer for 5 classes
Compiling the model...
Model built in: 5.067574977874756 seconds 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_2[0][0]']                
     